## 1. Imports

In [186]:
import numpy as np
import math
import random
import networkx as nx
from networkx import components
from scipy.linalg import expm
from scipy.linalg import sqrtm
from tqdm import tqdm
import matplotlib.pyplot as plt

from alt_Frozen_Network_class import alt_Frozen_Network as alt_Frozen_Network_class
from Genetic_Class import Genetic_Evolution

## 2. Settings

In [187]:
##########################################
#number of networks I wish to consider
pop_size = 200
##########################################
##########################################
#network parameters
##########################################
# number of nodes
n = 10

ts = np.linspace(0 , 0.5 , 6)

In [188]:
def kron(*matrices):
    result = np.array([[1]])
    for matrix in matrices:
        result = np.kron(result, matrix)
    return result

## 3. Target Setup

In [189]:

def F(n , prob):
    return nx.erdos_renyi_graph(n , prob)

##########################################
#target
##########################################
prob = 0.1
target = F(n , prob)


def adjacency_matrix_target():
    return [np.array(nx.to_numpy_array(target).astype("complex")) for _ in range(n)]

Starting state setup

In [190]:
k = 1
rho0 = np.zeros((n , 1))
rho0[k] = 1
state = kron(rho0 , np.conjugate(rho0.T))

##########################################
# Adding the sink to the target list
##########################################
def target_sink_func():
    mat_list = adjacency_matrix_target()
    for i in range(n):
        mat_list[i][i , i] -= 1j
    return mat_list
target_sink = target_sink_func()

In [191]:

#target
def evolution_target(t):
    #expmatrix = expm(complex(0 , -1) * target_sink()[i] * t)
        return [
            
                1-(np.trace
                    ((expm(complex(0 , -1) * target_sink[i] * t))
                    @ state
                    @ (np.conjugate(expm(complex(0 , -1) * target_sink[i] * t).T))
                  )
                ).real
            
            for i in range(n)
            ]
def target_data():
    data = []
    for t in ts:
        data.append(evolution_target(t))
    return np.array(data)


## 4. Population Setup

In [192]:
def alt_genetic_distance(pop , start_index = 0):
    dist = []
    for i in range(start_index , pop_size):
        dist.append([alt_Frozen_Network_class(pop[i] , target_data() , ts).calculate_distance() , pop[i]])
    dist = sorted(dist , key = lambda x: x[0])
    return [d[0] for d in dist] , [d[1] for d in dist]

def fit_half(pop):
    return pop[:pop_size // 2]

def pop(pop_size, n):
    return [nx.to_numpy_array(F(n, random.random())).astype("complex") for _ in range(pop_size)]

## 5. Genetic Algorithm

In [193]:
targets = []
combined_distances = []
combined_best_individual = []
combined_number_of_connections = []
combined_new_pop_set = []
combined_best_distance = []
combined_mean_distance = []
combined_fittest_individual = []
combined_connections = []
combined_degree_distribution = []
combined_found_it = []

In [ ]:
extreme_mutation_count = 0

for matthew in range(100):

    ##########################################
    #target
    ##########################################
    target = F(n , np.random.rand())
    targets.append(target)

    k = 1
    rho0 = np.zeros((n , 1))
    rho0[k] = 1
    state = kron(rho0 , np.conjugate(rho0.T))

    ##########################################
    # Adding the sink to the target list
    ##########################################
    target_sink = target_sink_func()

    population = pop(pop_size, n)

    
    
    alt_distances = [alt_genetic_distance(population)[0]]

    new_pop = Genetic_Evolution(fit_half(alt_genetic_distance(population)[1]) , alt_distances[0][:len(alt_distances[0])//2]).new_population()

    distances = []
    distances_ML = []
    best_individual = []
    number_of_connections = []
    new_pop_set = []
    best_distance = []
    mean_distance = []
    fittest_individual = []
    connections = []
    degree_distribution = []
    found_it = []

    for j in tqdm(range(100)):

        fit_individual = []
        if j == 0:
            new_distances, new_pop = alt_genetic_distance(new_pop)
        else:
            new_distances, new_population = alt_genetic_distance(new_pop , start_index = pop_size // 4)

            # Combine distances and population from current and new generations
            combined = list(zip(distances[-1][:pop_size // 4] + new_distances, distances[-1][:pop_size // 4] + new_distances, new_pop[:pop_size // 4] + new_population))
            # Sort by distances
            combined.sort(key=lambda x: x[0])
            # Unpack sorted distances and population
            new_distances, new_distance, new_pop = zip(*combined)
            # Convert to lists (if needed)
            new_distances = list(new_distances)
            new_pop = list(new_pop)

        distances.append(new_distances)
        best_individual.append(new_pop[0])

        number_of_connections.append(np.array([np.sum(np.real(adjacency_matrix_target()[0]))/2 , np.sum(np.real(new_pop[0]))/2 , np.sum(np.real(new_pop[1]))/2 , np.sum(np.real(new_pop[2]))/2]))
        new_pop_set.append(np.array(new_pop[:pop_size // 4]))
        degree_distribution.append([degree for node , degree in sorted(nx.degree(nx.from_numpy_array(np.real(new_pop[0]))), key=lambda x: x[1], reverse=True)])
        #Calculating the best and mean distances
        best_distance = min(distances[j])
        mean_distance = np.mean(new_distances)
        fittest_individual = new_pop[0]
        connections = np.array([np.sum(np.real(adjacency_matrix_target()[0]))/2 , np.sum(np.real(new_pop[0]))/2 , np.sum(np.real(new_pop[1]))/2 , np.sum(np.real(new_pop[2]))/2 , np.sum(np.real(new_pop[3]))/2])
        degree_dist = [degree for node , degree in sorted(nx.degree(nx.from_numpy_array(np.real(new_pop[0]))), key=lambda x: x[1], reverse=True)]

        # what to do if the algorithm finds the fittest individual
        # else rerun code
        print('Best Dist: ', min(distances[j]))
        print('Average Dist: ', np.mean(distances[j]))
        print('Best Adj Error: ', np.sum(np.abs(new_pop[0] - adjacency_matrix_target()[0])))
        print('Average Adj Dist Error: ', np.mean([np.sum(np.abs(p - adjacency_matrix_target()[0])) for p in new_pop]))
        if min(distances[j]) == 0:
            for i, dist in enumerate(distances[j]):
                if dist == 0:
                    fit_individual.append(new_pop[i])
            print("fit individual found")
            break
        else:
            new_pop = Genetic_Evolution(fit_half(alt_genetic_distance(new_pop)[1]) , distances[j][:len(distances[j])//2]).new_population()
        # implementing extreme mutations if population scores aren't improving
        if j > 5:
            print("extreme")
            if (new_pop_set[j-5] == new_pop_set[j]).all():
                new_pop = Genetic_Evolution(fit_half(alt_genetic_distance(new_pop)[1]) , distances[j][:len(distances[j])//2]).EXTREME_new_population()
                # if this is performed we add +1 to the extreme_mutation_count
                extreme_mutation_count += 1
                # printing when this if performed
                
                
        # the case of reaching 3 in extreme_mutation_count means we should inject a new population of individuals
        if extreme_mutation_count == 5:
            # inject a new population of individuals and keeping old unmutated parents
            new_pop = new_pop[:int(pop_size * 0.75)] + pop(pop_size//4 , n)
            print("new injection")
            # resetting the extreme_mutation_count to zero
            extreme_mutation_count = 0
            

    else:
        print("No fit individual found within iteration limit")
    combined_distances.append(distances)
    combined_best_individual.append(best_individual)
    combined_number_of_connections.append(number_of_connections)
    combined_new_pop_set.append(new_pop_set)
    combined_best_distance.append(best_distance)
    combined_mean_distance.append(mean_distance)
    combined_fittest_individual.append(fittest_individual)
    combined_connections.append(connections)
    combined_degree_distribution.append(degree_distribution)
    combined_found_it.append(found_it)

  0%|          | 0/100 [00:00<?, ?it/s]

Best Dist:  0.40782708789551836
Average Dist:  0.5749540292236636
Best Adj Error:  30.0
Average Adj Dist Error:  32.25


  1%|          | 1/100 [00:03<05:09,  3.13s/it]

Best Dist:  0.40782708789551836
Average Dist:  0.5283650067399435
Best Adj Error:  30.0
Average Adj Dist Error:  28.55


  2%|▏         | 2/100 [00:06<04:56,  3.03s/it]

...

extreme
Best Dist:  0.13094305300328077
Average Dist:  0.20047472834394675
Best Adj Error:  26.0
Average Adj Dist Error:  27.19


 23%|██▎       | 23/100 [03:31<06:00,  4.69s/it]

extreme
Best Dist:  0.12942442826675155
Average Dist:  0.15242902183241147
Best Adj Error:  20.0
Average Adj Dist Error:  26.57


 24%|██▍       | 24/100 [03:34<05:21,  4.24s/it]

extreme


## 6. Physical Properties

In [62]:
target_evolution = []
fittest_evolution = []

def state_evolution(matrix , t):
        return (expm(complex(0 , -1) * matrix * t)) @ state @ (np.conjugate(expm(complex(0 , -1) * matrix * t).T))
                
target_states_evolution = []
for i in range(len(targets)):
    for t in ts:
        target_states_evolution.append(state_evolution(nx.to_numpy_array(targets[i]) , t))

fittest_states_evolution = []
for i in range(len(combined_best_individual)):
     for t in ts:
          fittest_states_evolution.append(state_evolution(combined_best_individual[i][-1] , t))

Fidelity

In [67]:
def fidelity(mat1 , mat2):
    return np.real((np.trace(sqrtm(sqrtm(mat1) @ mat2 @ sqrtm(mat1))))**2)

fidelity_tab = []
for i in range(len(target_states_evolution)):
    fidelity_tab.append(fidelity(target_states_evolution[i] , fittest_states_evolution[i]))

Coherence

In [127]:
def coherence(mat1 , mat2):
     return np.abs(np.abs(np.sum(mat1) - np.trace(mat1)) - np.abs(np.sum(mat2) - np.trace(mat2)))

coherences_tab = []
for i in range(len(target_states_evolution)):
     coherences_tab.append(coherence(target_states_evolution[i] , fittest_states_evolution[i]))

Population

In [177]:
def populations(mat1 , mat2):
    pop = []
    for j in range(len(mat1)):
        pop.append(np.abs(np.real(mat1[j][j]) - np.real(mat2[j][j])))
    return pop

population_tab = []
for i in range(len(target_states_evolution)):
    population_tab.append(np.sum(populations(target_states_evolution[i] , fittest_states_evolution[i])))